## 1) Сбор корпуса
Для корпуса я собрала все стихотворения Блока, сохранила в словарь.

In [15]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd
import pickle

In [4]:
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alinazabolotskaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
URL = "https://slova.org.ru/blok/"
HEADERS = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36", "accept" : "*/*"} 
HOST = 'https://slova.org.ru'
PATH = r'Documents\avtobreya_project.csv' 
ua = UserAgent(verify_ssl=False)
session = requests.session()
author = "Александр Блок"

In [6]:
def get_poem(url):
    html = requests.get(url, headers=HEADERS) 
    print(f"{html.status_code} -- {url}")
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, 'html.parser')
        data = []
        text = soup.find('div', class_='grid-col-3').find_next('pre').get_text()
        for em in soup.find('div', class_='grid-col-3').find_next('pre').find_all('em'):
            text = text.replace(em.get_text(), '')
    return text
        

In [7]:
def poem_walker(url):
    html = requests.get(url, headers=HEADERS) 
    print(f"{html.status_code} -- {url}")
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, 'html.parser')
        data = {}
        for item in soup.find('div', class_='grid-col-1').find_all('a'):
            name = item.get_text()
            link = item.get('href')
            poem = get_poem(HOST+link)
            data.update({name: poem})
   
    return data

In [8]:
poems = poem_walker(URL)

200 -- https://slova.org.ru/blok/
200 -- https://slova.org.ru/blok/aviator/
200 -- https://slova.org.ru/blok/iavidelmrak/
200 -- https://slova.org.ru/blok/antverpen/
200 -- https://slova.org.ru/blok/barka_zhizni_vstala/
200 -- https://slova.org.ru/blok/begutnevernye/
200 -- https://slova.org.ru/blok/bezveryvboga/
200 -- https://slova.org.ru/blok/bezmenjab/
200 -- https://slova.org.ru/blok/bezradostnyevskhodjat/
200 -- https://slova.org.ru/blok/beloynochiiu/
200 -- https://slova.org.ru/blok/belyjkonchut/
200 -- https://slova.org.ru/blok/blagoslovlyaiu_vse/
200 -- https://slova.org.ru/blok/blazhennoty/
200 -- https://slova.org.ru/blok/blesnulo_v_glazakh/
200 -- https://slova.org.ru/blok/bozhekakzhizn/
200 -- https://slova.org.ru/blok/bolotistim/
200 -- https://slova.org.ru/blok/bolotnyechertenjatki/
200 -- https://slova.org.ru/blok/bolotnyjpopik/
200 -- https://slova.org.ru/blok/bolotoglubokaja/
200 -- https://slova.org.ru/blok/brozhuvstenakh/
200 -- https://slova.org.ru/blok/budetden/
2

In [9]:
copy = poems

## 2) Деление на предложения
Добавляю в словарь уровень с предложениями.

In [10]:
def sentencize(text):
    sentences = nltk.sent_tokenize(text)
    result = {}
    for sent in sentences:
        sent = sent.replace("\n", " ")
        sent = ' '.join(tokenizer.tokenize(sent))
        result.update({sent: tokenizer.tokenize(sent)})
        
    return result

Оставляю только стихотворения, подходящие по регламенту (от 100 слов)

In [11]:
new = {}
for poem in poems:
    if len(poems[poem].split()) > 99:
        sents = sentencize(poems[poem])
        for sent in sents:
            new.update({poem: sentencize(poems[poem])})

## 3) Морфологический разбор
Добавляю в словарь уровень разбора. Для каждого слова мне нужна только лемма, словоформа и часть речи.

In [12]:
from pymystem3 import Mystem
m = Mystem(disambiguation=True)

In [13]:
morphed = {}
pos_list = set()
for poem in new:
    for sent in new[poem]:
        morphed_sent = []
        ana = m.analyze(sent)
        for word in ana:
            text = word['text']
            if text != " " and text != "\n":
                if "analysis" in word.keys() and word['analysis'] != []:
                    pos = word['analysis'][0]['gr'].split('=')[0].split(',')[0]
                    lex = word['analysis'][0]['lex']
                    pos_list.add(pos)
                else:
                    pos = None
                    lex = text
                anadict = {'text': text, 'lex': lex, 'pos': pos}
                morphed_sent.append(anadict)
        new[poem].update({sent: morphed_sent})

In [18]:
with open(PATH, 'wb') as f:
    pickle.dump(new, f)

In [19]:
with open(PATH, 'rb') as f:
    new = pickle.load(f)

## 4) Подготавливаем функции для поиска

При поиске слова без кавычек я хочу учитывать омонимию, поэтому использую в этот раз пайморфи для получения нескольких разборов.

In [15]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Так как по регламенту слов в поиске может быть до трех, можно позволить себе написать отдельные функции.

In [17]:
def get_first_word(req):
    incl_sent = []
    for poem in new:
        for sent in new[poem]:
            mysent = new[poem][sent]
            for word in new[poem][sent]:
                iword = new[poem][sent].index(word)
                if req[0].startswith("\"") and req[0].endswith("\""):
                    if word['text'].lower() == req[0].strip("\"").lower():
                        incl_sent.append([sent, new[poem][sent], iword, poem])
                elif req[0].startswith("\"") == False and req[0].endswith("\"") == False:
                    if req[0] in pos_list:
                        if word['pos'] == req[0]:
                            incl_sent.append([sent, new[poem][sent], iword, poem])
                    elif req[0] not in pos_list:
                        if "+" in req[0]:
                            if word['text'].lower() == req[0].split("+")[0].lower() and word['pos'] == req[0].split("+")[1]:
                                incl_sent.append([sent, new[poem][sent], iword, poem]) #ищет слофоформу + часть речи
                        else:
                            for a in morph.parse(req[0]):
                                lexeme = a.normal_form
                                if word['lex'] == lexeme:
                                    if [sent, new[poem][sent], iword, poem] not in incl_sent:
                                        incl_sent.append([sent, new[poem][sent], iword, poem])
                                    for i in incl_sent:
                                        if sent not in i:
                                            incl_sent.append([sent, new[poem][sent], iword, poem])
                else:
                    print('Неверный формат поиска.')
    return incl_sent

In [18]:
def get_second(req, res):
    incl_sent = []
    for sent in res:
        iword = sent[2] + 1
        first = sent[1][sent[2]]
        if len(sent[1]) > sent[2]+1:
            word = sent[1][sent[2]+1]
            if req[1].startswith("\"") and req[1].endswith("\""):
                if word['text'].lower() == req[1].strip("\"").lower():
                    incl_sent.append(sent)
            elif req[1].startswith("\"") == False and req[1].endswith("\"") == False:
                if req[1] in pos_list:
                    if word['pos'] == req[1]:
                        incl_sent.append(sent)
                elif req[1] not in pos_list:
                    if "+" in req[1]:
                        if word['text'].lower() == req[1].split("+")[0].lower() and word['pos'] == req[1].split("+")[1]:
                            incl_sent.append(sent) #ищет слофоформу + часть речи
                    else:
                        for a in morph.parse(req[1]):
                            lexeme = a.normal_form
                            if word['lex'] == lexeme:
                                if sent not in incl_sent:
                                    incl_sent.append(sent)
                                
            else:
                print('Неверный формат поиска.')
    return incl_sent

In [19]:
def get_last(req, res):
    incl_sent = []
    for sent in res:
        iword = sent[2] + 2
        first = sent[1][sent[2] + 1]
        if len(sent[1]) > sent[2]+2:
            word = sent[1][sent[2]+2]
            if req[2].startswith("\"") and req[2].endswith("\""):
                if word['text'].lower() == req[2].strip("\"").lower():
                    incl_sent.append(sent)
            elif req[2].startswith("\"") == False and req[2].endswith("\"") == False:
                if req[2] in pos_list:
                    if word['pos'] == req[1]:
                        incl_sent.append(sent)
                elif req[2] not in pos_list:
                    if "+" in req[2]:
                        if word['text'].lower() == req[2].split("+")[0].lower() and word['pos'] == req[1].split("+")[1]:
                            incl_sent.append(sent) #ищет слофоформу + часть речи
                    else:
                        for a in morph.parse(req[2]):
                            lexeme = a.normal_form
                            if word['lex'] == lexeme:
                                if sent not in incl_sent:
                                    incl_sent.append(sent)
                                
            else:
                print('Неверный формат поиска.')
    return incl_sent

In [20]:
def search(req):
    output = []
    if len(req.split()) == 1:
        result = get_first_word(req.split())
    elif len(req.split()) == 2:
        res1 = get_first_word(req.split())
        result = get_second(req.split(), res1)
    elif len(req.split()) == 3:
        res1 = get_first_word(req.split())
        res2 = get_second(req.split(), res1)
        result = get_last(req.split(), res2)
    for i in result:
        output.append(link, author, i[-1], i[0])
    return output

## 5) Поиск

In [368]:
search("пил+V я \"кровь\"")

['И пил я кровь из плеч благоуханных И был напиток душен и смолист Но не кляни повествований странных О том как длился непонятный сон Из бездн ночных и пропастей туманных К нам доносился погребальный звон Язык огня взлетел свистя над нами Чтоб сжечь ненужность прерванных времен']

In [ ]:
search("кровь")

In [ ]:
search(input())

плечи


In [22]:
"fhfhk".split()

['fhfhk']